In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


### <b>문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드</b>
<b>문제 설명</b><br>
: 콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("==> 1. 문서 로딩 → 콘텐츠분쟁해결 사례집 PDF 읽기...")
loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

print("==> 2. 문서 분할 → 법률 사례별로 청크 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        # 법률 사례 특성상 더 큰 청크 사용
    chunk_overlap=300,      # 사례 맥락 보존을 위한 중복
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ] # 법률 문서 구조에 맞는 구분자
)

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print("==> 3. 벡터화 → 법률 용어 임베딩으로 변환")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  # 한국어 법률 용어에 적합한 모델
    dimensions=1536
)

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f"  FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

print("==> 5. 검색 → 유사 분쟁사례 검색기 설정")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 5개 관련 사례 검색
)
print("  Retriever 설정 완료")

print("==> 6. 생성 → 법률 자문 LLM 설정")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,  # 법률 조언은 정확성이 중요하므로 낮은 온도
    max_tokens=2000
)

# 법률 자문 전용 프롬프트
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

질문: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print("  법률 자문 프롬프트 설정 완료")

print("\n==> 7. QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("  콘텐츠분쟁해결 RAG 시스템 구축 완료!")

# 테스트용 분쟁 상황들
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "게임 계정이 불법 프로그램 사용 의혹으로 영구 정지되었는데, 사용한 적이 없습니다. 어떻게 대응해야 하나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

print("\n" + "=" * 70)
print("                   콘텐츠분쟁해결 RAG 시스템 테스트")
print("=" * 70)

# 질문 및 답변 실행
for i, question in enumerate(test_questions, 1):
    print(f"\n【분쟁사례 테스트 {i}/5】")
    print(f" 상담 내용: {question}")
    print(" 관련 사례 검색 및 법률 조언 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 법률 자문:")
    print("-" * 60)
    print(answer)
    
    # 참조 사례 정보
    print(f"\n 참조 분쟁사례:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:100].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 50)

print("\n RAG 시스템 테스트 완료!")
print(" 실제 분쟁 상황에서 이 시스템을 활용하여 관련 사례와 법적 근거를 빠르게 찾을 수 있습니다.")

* API 코드 얻으면 다시 해보기 !!!!!!